In [2]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import re

In [3]:
#Funcion para arrancar el webdriver
def iniciar_wd(headless=False):
    options=uc.ChromeOptions()
    options.add_argument('--password-store=basic')
    options.add_experimental_option(
        'prefs',
        {'credentials_enable_service':False,
         'profile.password_manager_enabled':False}
    )
    if headless:
        options.add_argument('--headless')

    #Iniciamos el driver
    driver=uc.Chrome(options=options,headless=headless,log_level=3)
    driver.maximize_window()
    
    return driver

In [4]:
def ext_info(driver,url):
    driver.get(url)
    time.sleep(3)
    tit=desc_s=info=evaluaciones=precio=limp=vera=lleg=com=ub=cal=serv=local=''
    
    html=driver.page_source
    soup=BeautifulSoup(html, 'html.parser')
    try:
        tit=soup.find('h1').text
    except:
        print('ERROR al obtener el Titulo')
    
    try:
        desc_s=soup.find('h2').text
    except:
        print('Error al obtener la descripcion')
    
    try:
        info=soup.find('ol').text
    except:
        print('Error al obtener la informacion')

    try:
        ev=soup.find_all('span')
        pattern_eval=r'^[0-9]+\s(evaluaciones)$'
        pattern_price=r'[0-9]+\s(€)\s$'
        evalu=price=0
        for e in ev:
            if len(re.findall(pattern_eval,e.text))>0 and evalu==0:
                evaluaciones=e.text
                evalu+=1
                
            if len(re.findall(pattern_price,e.text))>0 and price==0:
                precio=e.text
                price+=1

            if price==1 and evalu==1:
                break
        
        divs=soup.find_all('div')
        pat_limp=r'^(Limpieza)[0-9],[0-9]$'
        pat_ver=r'^(Veracidad)[0-9],[0-9]$'
        pat_lleg=r'^(Llegada)[0-9],[0-9]$'
        pat_com=r'^(Comunicación)[0-9],[0-9]$'
        pat_ub=r'^(Ubicación)[0-9],[0-9]$'
        pat_cal=r'^(Calidad)[0-9],[0-9]$'

        for d in divs:
            if len(re.findall(pat_limp,d.text))>0:
                limp=d.text
            
            elif len(re.findall(pat_ver,d.text))>0:
                vera=d.text
            elif len(re.findall(pat_lleg,d.text))>0:
                lleg=d.text
            elif len(re.findall(pat_com,d.text))>0:
                com=d.text
            elif len(re.findall(pat_ub,d.text))>0:
                ub=d.text
            elif len(re.findall(pat_cal,d.text))>0:
                cal=d.text
            else:
                pass

    except:
        print('No tiene evaluaciones')
    
    sps=soup.find_all('section')

    se=lo=0
    for s in sps:
        if s.text.find('¿Qué hay en este alojamiento?')!=-1 and se==0:
            serv=s.text[len('¿Qué hay en este alojamiento?'):]
            se+=1
        
        if s.text.find('¿Dónde me voy a quedar?')!=-1 and lo==0:
            local=s.text[len('¿Dónde me voy a quedar?'):]
            lo+=1

        if se==1 and lo==1:
            break
        
    return tit,desc_s,info,evaluaciones,precio,limp,vera,lleg,com,ub,cal,serv,local
    

In [9]:
import json

with open('urls.json','r') as jsonfile:
    data=json.load(jsonfile)

driver=iniciar_wd(headless=True)
claves = ["ids", "titulos", "desc_sen", "informacion", "eval", "tipo", "precios", "enlaces", 
          "limpieza", "veracidad", "llegada", "comunicacion", "ubicacion", "calidad", 
          "servicios", "localizacion"]
datos = {clave:[] for clave in claves}

for k in data.keys():
    for url in data[k]:
        
        a,b,c,d,f,g,h,i,j,l,m,n,o=ext_info(driver,url)
        nuevo_id = hash(url)
        nuevos_datos = [nuevo_id, a, b, c, d, k, f, url, g, h, i, j, l, m, n, o]

        for clave, dato in zip(claves, nuevos_datos):
            datos[clave].append(dato)

driver.close()

KeyboardInterrupt: 

In [10]:
result=pd.DataFrame(datos,columns=claves)
result.shape

(12, 16)

In [ ]:
result.to_csv('DatasetAirbnb.csv',index=False)